In [11]:
#NLP libraries
import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim
import findspark
import pyspark.sql.functions as psf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import re

from pyspark.sql import *
from pyspark.sql.types import *

findspark.init()

spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.session.timeZone', 'UTC')
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [2]:
nlp = spacy.load('en')

In [27]:
films = spark.read.parquet("parquets/films2.parquet")

In [28]:
films.columns

['tconst', 'num_subtitles', 'year', 'blocks', 'subtitle_mins', 'subtitles']

In [29]:
r = films.select('subtitles').take(1)
r[0]

[Row(subtitles=[['As', 'a', 'kid', 'growing', 'up', 'in', 'the', 'ghetto', ',', 'one', 'of', 'the', 'things', 'I', 'wanted', 'most', 'was', 'not', 'money', ',', 'it', 'was', 'fame', '.'], ['I', 'wanted', 'to', 'be', 'known', '.'], ['I', 'want', 'the', 'people', 'to', 'say', ',', '"', 'Hey', ',', 'there', 'goes', 'O.J.', '"'], ['You', "'re", 'approaching', 'five', 'years', 'now', 'at', 'Lovelock', '.'], ['Tell', 'us', 'about', 'your', 'work', 'assignments', '.'], ['How', 'have', 'you', 'occupied', 'your', 'time', 'for', 'the', 'past', 'five', 'years', '?'], ['Uh', ',', 'when', 'I', 'first', 'came', 'here', ',', 'I', 'was', 'a', 'porter', ',', 'which', 'comprised', 'of', 'cleaning', 'things', 'in', 'the', 'unit', 'that', 'I', 'was', 'in', ',', 'and', 'basically', 'after', 'a', 'relatively', 'short', 'period', 'of', 'time', ','], ['I', 'started', 'working', 'as', 'a', 'gym', 'worker', '.'], ['I', 'start', 'each', 'day', ',', 'um', ',', 'disinfecting', ',', 'uh', ',', 'the', 'workout', 'eq

In [3]:
df_text = spark.read.parquet("parquets/filmtext.parquet")

In [4]:
df_text = df_text.select('tconst','text')

In [5]:
r = df_text.take(1)
r

[Row(tconst='tt5275892', text=' As a kid growing up in the ghetto, one of the things I wanted most was not money, it was fame.\n I wanted to be known.\n I want the people to say," Hey, there goesO.J."\n You\'re approaching five years now at Lovelock.\n Tell us about your work assignments.\n How have you occupied your time for the past five years?\n Uh, when I first came here, I was a porter, which comprised of cleaning things in the unit that I was in, and basically after a relatively short period of time,\n I started working as a gym worker.\n I start each day, um, disinfecting, uh, the workout equipment in the gym, mopping floors with the other, uh, group of us that work in the gym.\n Uh, I\'ve coached teams, um, uh, since I\'ve been here.\n Uh, and I like to say that we won the championship, and we were old guys, a totally mixed group of players.\n I didn\'t play, I just coached.\n I do see that in 1994 you were arrested at the age of 46.\nWe-\n- We\'re talking about this case?\n No

In [6]:
training_text = r[0][1]

In [7]:
print(training_text)

 As a kid growing up in the ghetto, one of the things I wanted most was not money, it was fame.
 I wanted to be known.
 I want the people to say," Hey, there goesO.J."
 You're approaching five years now at Lovelock.
 Tell us about your work assignments.
 How have you occupied your time for the past five years?
 Uh, when I first came here, I was a porter, which comprised of cleaning things in the unit that I was in, and basically after a relatively short period of time,
 I started working as a gym worker.
 I start each day, um, disinfecting, uh, the workout equipment in the gym, mopping floors with the other, uh, group of us that work in the gym.
 Uh, I've coached teams, um, uh, since I've been here.
 Uh, and I like to say that we won the championship, and we were old guys, a totally mixed group of players.
 I didn't play, I just coached.
 I do see that in 1994 you were arrested at the age of 46.
We-
- We're talking about this case?
 No, the age at first arrest.
 How old were you, first

In [30]:
from nltk.corpus.reader.aligned import AlignedCorpusReader
reader = AlignedCorpusReader

In [35]:
# wirtting the file into .txt to use reader 

flow = open('test.txt','w')

flow.write(training_text)

flow.close()

In [38]:
reader('test.txt',[0])

<AlignedCorpusReader in '/Users/guerra/ADA/adamax/test.txt'>

In [24]:
print(re.match("^[\\[\"]$", "\""))

<_sre.SRE_Match object; span=(0, 1), match='"'>


In [8]:
def get_chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [9]:
# Get the chunks again (into smaller chunks)
# dictionary of films (tconst: index)
book_id = {f: n for n,f in enumerate(our_books.fileids())} 
chunks = list()
chunk_class = list() # this list contains the original book of the chunk, for evaluation

limit = 60 # how many chunks total
size = 50 # how many sentences per chunk/page

for f in our_books.fileids():
    sentences = our_books.sents(f)
    print(f)
    print('Number of sentences:', len(sentences))
    
    # create chunks
    chunks_of_sents = [x for x in get_chunks(sentences,size)] # this is a list of lists of sentences, which are a list of tokens
    chs = list()
    
    # regroup so to have a list of chunks which are strings
    for c in chunks_of_sents:
        grouped_chunk = list()
        for s in c:
            grouped_chunk.extend(s)
        chs.append(" ".join(grouped_chunk))
    print("Number of chunks:",len(chs),'\n')
    
    # filter to the limit, to have the same number of chunks per book
    chunks.extend(chs[:limit])
    chunk_class.extend([book_id[f] for _ in range(len(chs[:limit]))])

NameError: name 'our_books' is not defined

In [ ]:
STOPWORDS = spacy.lang.en.stop_words.STOP_WORDS

processed_docs = list()
for doc in nlp.pipe(chunks, n_threads=5, batch_size=10):

    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list and keep only words of length 3 or more.
    doc = [token for token in doc if token not in STOPWORDS and len(token) > 2]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)
docs = processed_docs
del processed_docs

In [ ]:
# Add bigrams too
from gensim.models.phrases import Phrases

# Add bigrams to docs (only ones that appear 15 times or more).
bigram = Phrases(docs, min_count=15)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]
#MmCorpus.serialize("models/corpus.mm", corpus)

print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(corpus))

In [ ]:
# models
from gensim.models import LdaMulticore
params = {'passes': 10, 'random_state': seed}
base_models = dict()
model = LdaMulticore(corpus=corpus, num_topics=4, id2word=dictionary, workers=6,
                passes=params['passes'], random_state=params['random_state'])

In [ ]:
model.show_topics(num_words=5)

In [ ]:
model.show_topic(1,20)

In [ ]:
sorted(model[corpus[0]],key=lambda x:x[1],reverse=True)

In [ ]:
# plot topics
data =  pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(data)